In [1]:
import sys
import pickle
import pandas as pd
import json
import numpy as np
import requests
from datetime import datetime, timedelta
import pytz
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient, Point
import os
from dotenv import load_dotenv

sys.path.insert(1, "../library")  # noqa: E402
import database_helper  # noqa: E402

In [ ]:
load_dotenv()

In [ ]:
with open("selected_columns.pkl", "rb") as f:
    selected_columns = pickle.load(f)
    print(selected_columns)

strings_to_exclude = ["icon", "moon", "warning"]
selected_columns = [
    item
    for item in selected_columns
    if not any(substring in item for substring in strings_to_exclude)
]

strings_to_include = ["days_0", "10838"]
selected_columns = [
    item
    for item in selected_columns
    if all(substring in item for substring in strings_to_include)
]

print(str(len(selected_columns)) + " Features selected")
df = database_helper.query_data(
    field_list=selected_columns, start_time="2025-01-01T12:00:00Z"
)

In [ ]:
df.set_index("_time", inplace=True)
df = df.select_dtypes(include="float64")
df.interpolate(inplace=True)
df = df.resample("h").mean()
df.reset_index(inplace=True)
date_time = pd.to_datetime(df.pop("_time"), format="%d.%m.%Y %H:%M:%S")

display(df.head())
display(date_time[0:5])

In [5]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)
day = 24 * 60 * 60
year = (365.2425) * day

df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))

In [6]:
df = (df - df.mean()) / df.std()

In [ ]:
num_features = df.shape[1]
time_range = 24
df = df[:time_range]
print(df.shape, num_features, time_range)
df.reset_index(drop=True, inplace=True)
display(df.head())
data = json.dumps(
    {"signature_name": "serving_default", "instances": np.array(df).tolist()}
)
print(data)

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post(
    "http://localhost:8501/v1/models/waether-timeseries-forecasts:predict",
    data=data,
    headers=headers,
)
response = json.loads(json_response.text)
predictions = response["predictions"]
print(predictions)

In [ ]:
def generate_timestamps(n):
    # Aktuelle Zeit in Berliner Zeitzone (CET/CEST)
    now = datetime.now(pytz.timezone("Europe/Berlin"))
    print(now)
    timestamps = []
    # Erstelle 'n' Timestamps, die jeweils 1 Stunde auseinanderliegen
    for i in range(n):
        timestamp = now + timedelta(hours=i + 1)
        timestamps.append(timestamp.strftime("%Y-%m-%d %H:%M:%S%z"))
    return timestamps

In [ ]:
INFLUXDB_URL = "http://tig.influxdb.local"
INFLUXDB_TOKEN = os.getenv("INFLUXDB2_TOKEN")
INFLUXDB_ORG = os.getenv("INFLUXDB2_ORGANIZATION")
INPUT_BUCKET = os.getenv("INFLUXDB2_BUCKET")
OUTPUT_BUCKET = os.getenv("INFLUXDB2_ML_BUCKET")

client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
timestamps = generate_timestamps(len(predictions))

if predictions:
    for i, prediction in enumerate(predictions):
        point = Point("inference_result").time(timestamps[i])
        for y, field in enumerate(prediction):
            point.field(df.columns[y], float(prediction[y]))
        write_api.write(bucket=OUTPUT_BUCKET, org=INFLUXDB_ORG, record=point)
    print("Predictions written to InfluxDB")